In [1]:
# Preciso criar um diretorio chamado laudo e jogar os oficios para dentro do mesmo (arquivos.pdf)
! mkdir -p laudo
! cd laudo
from google.colab import files
uploaded = files.upload()

In [2]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# instalação de pacotes e bibliotecas

#!git clone https://www.github.com/camelot-dev/camelot
#!cd camelot
!pip install   "camelot-py[cv]"

!apt-get install  ghostscript

!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org  'PyPDF2<3.0'
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org  pypi.org xlwt

!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org  pdfminer.six

!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org  pdfreader

import camelot
from pdfminer.high_level import extract_text

import pandas as pd
import io
import glob

from pdfreader import PDFDocument

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.8/257.8 kB 23.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of camelot-py[cv] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 57.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs

In [3]:
# Use glob to get a list of all CSV/PDF files in the folder
path_colab = r'/content/laudo/*.pdf'
path_jupyter_laptop = r'C:\Users\basson\Downloads\DSA\python\laudo\*.pdf'
path_jupyter_desktop = r'C:\Users\y05r\Downloads\laudo\*.pdf'

files = glob.glob(path_colab)
#files = glob.glob(path_jupyter)
#files = glob.glob(path_jupyter_desktop)

# Loop through the files

for i in range(0, len(files), 1):
    print(f'posicao: {i} e valor:{files[i]} ')

posicao: 0 e valor:/content/laudo/Laudo_3047648.pdf 
posicao: 1 e valor:/content/laudo/Laudo_2628563_7_RI_26_BA_221110214.pdf 
posicao: 2 e valor:/content/laudo/Laudo_3069866_7_SEP_8D_RJS_230510101.pdf 
posicao: 3 e valor:/content/laudo/Laudo_3014818_9_RO_138D_RJS_230410093.pdf 
posicao: 4 e valor:/content/laudo/Laudo_2787644_8_SEP_5D_RJS_230110033.pdf 
posicao: 5 e valor:/content/laudo/Laudo_2628256_7_BUZ_33_RJS_22111210.pdf 
posicao: 6 e valor:/content/laudo/Laudo_2891654_8_LL_118D_RJS_230310070.pdf 
posicao: 7 e valor:/content/laudo/Laudo_3069378_7_SEP_6_RJS_230510100.pdf 


In [4]:
#

for file in files:
    with open(file, 'rb') as pdf:
        #doc = PDFDocument(pdf)
        #print(f'posicao: {file} e valor:{files[file]} ')
        # Lendo documento .pdf e extraindo as tabela com a biblioteca Camelot

          lista_tabelas = camelot.read_pdf(file,header=None,index=False,encoding="utf-8",pages="all")

        # Criando Dataframe após extração das tabelas do arquivo .pdf
          lista_tabelas = lista_tabelas[0].df[[1,0,2]].reset_index(drop=True)

        ### Iniciando o ETL da extração das tabelas ###

        # remover as linhas e colunas desnecessárias a extração das informações
          lista_tabelas = lista_tabelas.drop(0, axis='index')
          lista_tabelas = lista_tabelas.drop(1, axis='index')

        # Renomear as colunas para o padrão do documento
          lista_tabelas.rename(columns = {1:'Norma Referencia',0:'Tipo Dados',2:'Situacao BDIEP'}, inplace = True)

          lista_tabelas = lista_tabelas.loc[lista_tabelas['Situacao BDIEP'] == 'NÃO CONFORME']
          print("\n")

        # Extraindo texto do documento com a biblioteca Extract_text
          texto = extract_text(file, 'rb')

        # Criando Dataframe após extração das tabelas do arquivo .pdf
          texto = pd.read_fwf(io.StringIO(texto))
          texto = texto[16:22]

        # Iniciando o ETL da extração do texto

        # remover linhas e colunas desnecessárias a extração das informações

        # Renomear colunas para o padrão do documento
          texto.rename(columns = {'Agencia Nacional de Petróleo, Gás Natura e Biocombustíveis':' CABEÇALHO'}, inplace = True)
          texto = texto.T
          texto.rename(columns = {16:'Nº do Laudo de Reprovação', 17:'Data da Avaliação',18:'Nome do Poço',19:'Nº Cadastro do Poço',20:'Conclusão do Poço',21:'Empresa Operadora'}, inplace = True)

          concatenar = pd.concat([texto,lista_tabelas],ignore_index=True)
          concatenar.shape
          display(concatenar)
          print("\n")

          # Saida do dataframe para um planilha excel
          #concatenar.to_excel('file_name.xls')


,Nº do Laudo de Reprovação,Data da Avaliação,Nome do Poço,Nº Cadastro do Poço,Conclusão do Poço,Empresa Operadora,Norma Referencia,Tipo Dados,Situacao BDIEP
0,2305.1.0099,08/05/2023,7-MRO-6DB-RJS,74316029648,23/02/2021,Petrobras,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.1. Arquivos Auxiliares - Esquemas de Ferrame...,NÃO CONFORME


,Nº do Laudo de Reprovação,Data da Avaliação,Nome do Poço,Nº Cadastro do Poço,Conclusão do Poço,Empresa Operadora,Norma Referencia,Tipo Dados,Situacao BDIEP
0,2211.1.0214,22/11/2022,7-RI-26-BA,202400489900,01/07/1987,Petrobras,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Nº do Laudo de Reprovação,Data da Avaliação,Nome do Poço,Nº Cadastro do Poço,Conclusão do Poço,Empresa Operadora,Norma Referencia,Tipo Dados,Situacao BDIEP
0,2305.1.0101,11/05/2023,7-SEP-8D-RJS,74316029573,07/02/2021,Petrobras,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.5. Perfis Convencionais,NÃO CONFORME
2,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.9. Perfil Digital Processado,NÃO CONFORME


,Nº do Laudo de Reprovação,Data da Avaliação,Nome do Poço,Nº Cadastro do Poço,Conclusão do Poço,Empresa Operadora,Norma Referencia,Tipo Dados,Situacao BDIEP
0,2304.1.0093,24/04/2023,9-RO-138D-RJS,74281027283,31/12/2013,Petrobras,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.2. Arquivos Auxiliares - Dados Direcionais,NÃO CONFORME
2,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.4. Perfis Durante a Perfuração,NÃO CONFORME
3,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.6. Perfis Especiais - Teste de Formação a Cabo,NÃO CONFORME


,Nº do Laudo de Reprovação,Data da Avaliação,Nome do Poço,Nº Cadastro do Poço,Conclusão do Poço,Empresa Operadora,Norma Referencia,Tipo Dados,Situacao BDIEP
0,2301.1.0033,24/01/2023,8-SEP-5D-RJS,74316029619,05/11/2020,Petrobras,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.5. Perfis Convencionais,NÃO CONFORME
2,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.9. Perfil Digital Processado,NÃO CONFORME


,Nº do Laudo de Reprovação,Data da Avaliação,Nome do Poço,Nº Cadastro do Poço,Conclusão do Poço,Empresa Operadora,Norma Referencia,Tipo Dados,Situacao BDIEP
0,2211.1.0210,18/11/2022,7-BUZ-33-RJS,74316029460,13/03/2020,Petrobras,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.1. Arquivos Auxiliares - Esquemas de Ferrame...,NÃO CONFORME
2,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.4. Perfis Durante a Perfuração,NÃO CONFORME
3,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.9. Perfil Digital Processado,NÃO CONFORME


,Nº do Laudo de Reprovação,Data da Avaliação,Nome do Poço,Nº Cadastro do Poço,Conclusão do Poço,Empresa Operadora,Norma Referencia,Tipo Dados,Situacao BDIEP
0,2303.1.0070,13/03/2023,8-LL-118D-RJS,74316029530,05/05/2020,Petrobras,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.4. Perfis Durante a Perfuração,NÃO CONFORME
2,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.5. Perfis Convencionais,NÃO CONFORME


,Nº do Laudo de Reprovação,Data da Avaliação,Nome do Poço,Nº Cadastro do Poço,Conclusão do Poço,Empresa Operadora,Norma Referencia,Tipo Dados,Situacao BDIEP
0,10/05/2023,7-SEP-6-RJS,74316029528,30/05/2020,Petrobras,2. CONTROLE DE REMESSAS (USO INTERNO),NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,RANP n° 880/2022,3.9. Perfil Digital Processado,NÃO CONFORME


In [ ]:

import PyPDF2

with open(r'C:\Users\y05r\Downloads\laudo\Laudo_3047648.pdf', 'rb') as arquivo:
    leitor_pdf = PyPDF2.PdfReader(arquivo)

    # Obtém o número total de páginas do PDF

    total_paginas = len(leitor_pdf.pages)

    for numero_pagina in range(total_paginas):

        pagina = leitor_pdf.pages[numero_pagina]

        texto = pagina.extract_text()

        # Faça algo com o texto extraído da página
        print(f"Conteúdo da página {numero_pagina + 1}:")
        print(texto)
        print("\n")

        f = open("texto.txt", "a+")
        f.write("\n" + texto)
        f.close()

#display(texto_pagina0.split("\n"))

Conteúdo da página 1:
Agencia Nacional de Petróleo, Gás Natura e Biocombustíveis
Superintendência de Dados Técnicos
Banco de Dados de Exploração e Produção
Laudo de Avaliação de Dados de Poço
2305.1.0099 7-MRO-6DB-RJS 74316029648 23/02/2021 Petrobras1.1. Nº do Laudo 
de Reprovação:1.2. Data da 
Avaliação :1.3. Nome do 
Poço:1.4. Nº Cadastro do 
Poço:1.5. Conclusão do 
Poço:1.6. Empresa 
Operadora:1. CABEÇALHO
08/05/2023
2.3. Código de Cadastro da(s) 
Mídia(s) no BDEP:2.2. Data de Protocolo da(s) 
Remessa(s):2.1. Número de Protocolo da(s) 
Remessa(s):2. CONTROLE DE REMESSAS (USO INTERNO)
48610.202029/2021-13 370_2021 14/05/2021
48610.202029/2021-13 385_2021 17/05/2021
*Legenda da Situação dos Dados de Poço no BDEP:
NÃO RECEBIDO = Conjunto de dados adquiridos pela operadora, conforme Notificação de Perfilagem Realizada (NPR), mas não 
remetidos ao BDEP;
NÃO CONFORME = Conjunto de dados avaliados como não conformes. As não conformidades estão descritas neste laudo;
APROVADO = Conjunto de 

In [ ]:

import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
# from pdfminer.pdfpage

def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(pdf_path, 'rb') as file:
        page_content = []
        for page in PDFPage.get_pages(file):
            page_interpreter.process_page(page)

            text_page = fake_file_handle.getvalue()
            page_content.append(text_page)


        # Contém o conteúdo do pdf completo
        text = fake_file_handle.getvalue()

    # close open handles
    converter.close()
    fake_file_handle.close()

    if text:
        return text, page_content


extract_text_from_pdf(r'C:\Users\y05r\Downloads\laudo\Laudo_3047648.pdf')[0]

'Agencia Nacional de Petróleo, Gás Natura e BiocombustíveisSuperintendência de Dados TécnicosBanco de Dados de Exploração e ProduçãoLaudo de Avaliação de Dados de Poço2305.1.00997-MRO-6DB-RJS7431602964823/02/2021Petrobras1.1. Nº do Laudo de Reprovação:1.2. Data da Avaliação:1.3. Nome do Poço:1.4. Nº Cadastro do Poço:1.5. Conclusão do Poço:1.6. Empresa Operadora:1. CABEÇALHO08/05/20232.3. Código de Cadastro da(s) Mídia(s) no BDEP:2.2. Data de Protocolo da(s) Remessa(s):2.1. Número de Protocolo da(s) Remessa(s):2. CONTROLE DE REMESSAS (USO INTERNO)48610.202029/2021-13370_202114/05/202148610.202029/2021-13385_202117/05/2021*Legenda da Situação dos Dados de Poço no BDEP:NÃO RECEBIDO = Conjunto de dados adquiridos pela operadora, conforme Notificação de Perfilagem Realizada (NPR), mas não remetidos ao BDEP;NÃO CONFORME = Conjunto de dados avaliados como não conformes. As não conformidades estão descritas neste laudo;APROVADO = Conjunto de dados aprovados e carregados no banco de dados. 3. R

In [ ]:
search_for = "NÃO CONFORMIDADES" # What to search in the log lines...
line_num = 0 # Line number
lines_found = 0 # Quantity of lines found...
out_file = "saida.txt"
in_file = "texto.txt"

with open(out_file, 'w') as out_file:
    with open(in_file, "r") as in_file:
        for line in in_file:
            line_num += 1
            if search_for in line:
                lines_found += 1
                print("Found '{}' in line {}...".format(search_for, line_num))
                print("Found '{}' ".format(line))
                out_file.write(line)

        print("Found {} lines...".format(lines_found))

Found 'NÃO CONFORMIDADES' in line 50...
Found 'Itens: Não Conformidades:5. NÃO CONFORMIDADES REFERENTES À REMESSA DE DADOS , À ESTRUTURA DE DIRETÓRIOS E AO FORMATO E 
' 
Found 'NÃO CONFORMIDADES' in line 56...
Found '6. NÃO CONFORMIDADES REFERENTES AOS RELATÓRIOS E ARQUIVOS AUXILIARES
' 
Found 'NÃO CONFORMIDADES' in line 62...
Found '7. NÃO CONFORMIDADES REFERENTES AO CABEÇALHO DE PERFIS DE POÇO (DURANTE A PERFURAÇÃO , 
' 
Found 'NÃO CONFORMIDADES' in line 88...
Found 'Itens: Não Conformidades:8. NÃO CONFORMIDADES REFERENTES AO CORPO DE PERFIS DE POÇO (DURANTE A PERFURAÇÃO , 
' 
Found 'NÃO CONFORMIDADES' in line 113...
Found '9. NÃO CONFORMIDADES REFERENTES AO RODAPÉ DO PERFIL COMPOSTO E DO PERFIL DE ACOMPANHAMENTO 
' 
Found 5 lines...


In [ ]:
descproblema = ['Valor associado a menmonicos'],['Dados nao remetidos ao BDEP'],['Ausência de informação de perflagem conforme NPR '],['Valor inválido associado a mnemônicos '],['Cabeçalho '],['Valor inválido associado a mnemônicos'],['Ausência de informação de perflagem conforme NPR']
df4 = pd.DataFrame(descproblema,columns=['Descrição do Problema'])

df4 = df4.reset_index(drop=True)
df4.info()
#df4.rename(columns = {'16':'team_name', '17':'points_scored'}, inplace = True)

In [ ]:
! python -m spacy download pt_core_news_sm
import spacy

# Import the Matcher
from spacy.matcher import Matcher

import pt_core_news_sm
nlp = pt_core_news_sm.load()
nlp = spacy.load("pt_core_news_sm")

doc = nlp (texto)


# Initialize the Matcher with the shared vocabulary
matcher = Matcher(nlp.vocab)

# Create a pattern matching two tokens: "iPhone" and "X"
pattern = [{"TEXT": "Laudo"}, {"TEXT": "de"}]

# Add the pattern to the matcher
matcher.add("Não Conformidades_X_PATTERN", [pattern])

# Use the matcher on the doc
matches = matcher(doc)
print("Matches:", [doc[start:end].text for match_id, start, end in matches])

C:\ProgramData\Anaconda3\python.exe: No module named spacy


ModuleNotFoundError: No module named 'spacy'